In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
import zipfile
import os
import gdown

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

# Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  # Name of the downloaded file

# Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F
From (redirected): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F&confirm=t&uuid=228099df-8feb-43c0-b191-de0abb84252f
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [00:23<00:00, 167MB/s]


In [ ]:
#Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

#Specify the exact classes you want to include
target_classes = ['AMD', 'Normal_Fundus']

#Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

#Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='validation',
    shuffle=False
)


# Model definition
def build_ImprovedCNN():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        GlobalAveragePooling2D(),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(2, activation='softmax')  # Only 2 classes: AMD and Normal_Fundus
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define final model
model = build_ImprovedCNN()


# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
)

Found 18441 images belonging to 2 classes.
Found 2048 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


577/577 ━━━━━━━━━━━━━━━━━━━━ 122s 175ms/step - accuracy: 0.7441 - loss: 0.5702 - val_accuracy: 0.6836 - val_loss: 1.7417
Epoch 2/5
577/577 ━━━━━━━━━━━━━━━━━━━━ 86s 150ms/step - accuracy: 0.9533 - loss: 0.1329 - val_accuracy: 0.8604 - val_loss: 0.4703
Epoch 3/5
577/577 ━━━━━━━━━━━━━━━━━━━━ 86s 149ms/step - accuracy: 0.9663 - loss: 0.0871 - val_accuracy: 0.6919 - val_loss: 2.4302
Epoch 4/5
577/577 ━━━━━━━━━━━━━━━━━━━━ 87s 150ms/step - accuracy: 0.9733 - loss: 0.0762 - val_accuracy: 0.9785 - val_loss: 0.0721
Epoch 5/5
577/577 ━━━━━━━━━━━━━━━━━━━━ 86s 149ms/step - accuracy: 0.9754 - loss: 0.0631 - val_accuracy: 0.9917 - val_loss: 0.0268


In [4]:
model.save("AMD.keras")